Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
import re

import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import wordnet

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier , Pool
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Классы не сбаланасированны.<br>
Сами данные могут содержать спецсимволы, например **\n** или **\r**. <br>
Также слова могут быть соединены не только пробелами, но и другими символами<br>
Возможно стоит заменить все числовые строки на одно число, чтобы сделать акцент именно на словах<br>
В общем, следует почистить строки

In [5]:
#приведем все в нижний регистр
df['text'] = df['text'].str.lower()

In [6]:
def my_replace(data):
    # функция для рег.выражений
    return data.group(0)[0]

def get_lemm(text):
    """
    Подготив строки, токенизируем и проведем лемматизацию
    """
    text = re.sub(r'[_|#|\r|\t|\/]', ' ', text) # Заменим различные символы на пробел
    text = re.sub(r'\.', ' . ', text) # Добавим пробел вокруг точек
    text = re.sub(r'\,', ' , ', text) # Добавим пробел вокруг запятых
    text = re.sub(r'\n', ' . ', text) # Заменим переносы строк на точки
    text = re.sub(r'n\'t', ' not ', text) # например don't => do not 
    text = re.sub(r'\b[0-9]\w*|\w*[0-9]', ' 9999 ', text) # все слова содержащие числа будут заменены на число 9999
    text = re.sub(r'a{3,}|b{3,}|c{3,}|\
         d{3,}|e{3,}|f{3,}|\
         g{3,}|h{3,}|i{3,}|\
         j{3,}|k{3,}|l{3,}|\
         m{3,}|n{3,}|o{3,}|\
         p{3,}|q{3,}|r{3,}|\
         s{3,}|t{3,}|u{3,}|\
         v{3,}|w{3,}|x{3,}|\
         y{3,}|z{3,}', my_replace, text) # Если буква повторяется в слове 3 раз или более, то заменяем только на 1 повтор
                                         # Например nooooo => no
    word_list = nltk.word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(w) for w in word_list])

# Проверим
print(df.iloc[0]['text'])
get_lemm(df.iloc[0]['text'])

explanation
why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27


"explanation . why the edits made under my username hardcore metallica fan were reverted ? they were not vandalism , just closure on some gas after i voted at new york doll fac . and please do not remove the template from the talk page since i 'm retired now . 9999 . 9999 . 9999 . 9999"

In [7]:
%%time
df['text_lemma'] = df['text'].apply(get_lemm)
df.head()

CPU times: user 4min 55s, sys: 728 ms, total: 4min 56s
Wall time: 4min 58s


,text,toxic,text_lemma
0,explanation\nwhy the edits made under my usern...,0,explanation . why the edits made under my user...
1,d'aww! he matches this background colour i'm s...,0,d'aww ! he match this background colour i 'm s...
2,"hey man, i'm really not trying to edit war. it...",0,"hey man , i 'm really not trying to edit war ...."
3,"""\nmore\ni can't make any real suggestions on ...",0,`` . more . i ca not make any real suggestion ...
4,"you, sir, are my hero. any chance you remember...",0,"you , sir , are my hero . any chance you remem..."


Подготовим данные к обучению

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text_lemma'],
    df['toxic'], 
    test_size=0.2, 
    random_state=42
)

In [9]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=42)
    
    return features_upsampled, target_upsampled



### Баланс классов в итоге ухудшил модели

In [11]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords, min_df=5)

TfidfVectorizer обучим на тренировочной выборке

In [ ]:
corpus = X_train.values.astype('U')

In [ ]:
count_tf_idf.fit(corpus)

In [ ]:
tf_idf = count_tf_idf.transform(corpus)
tf_idf

Подготовим признаки для теста

In [ ]:
corpus_test = X_test.values.astype('U')

# 2. Обучение

### LogisticRegression

In [21]:
# обучите и протестируйте модель
model = LogisticRegression(random_state=42, class_weight='balanced')
model.fit(tf_idf, y_train)
y_pred = model.predict(tf_idf_test)
f1_score(y_test, y_pred)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7415310749533208

### CatBoostRegressor

In [22]:
%%time
model = CatBoostClassifier(
    custom_metric='F1', 
    iterations=500, 
    depth=4, 
    l2_leaf_reg=8,
    learning_rate=0.67, 
    random_seed=42,
    bagging_temperature=.5
)

model.fit(tf_idf, y_train, verbose=10)

0:	learn: 0.2960854	total: 228ms	remaining: 1m 53s
10:	learn: 0.1868755	total: 1.63s	remaining: 1m 12s
20:	learn: 0.1652825	total: 3.11s	remaining: 1m 10s
30:	learn: 0.1536957	total: 4.58s	remaining: 1m 9s
40:	learn: 0.1442116	total: 6.03s	remaining: 1m 7s
50:	learn: 0.1387072	total: 7.44s	remaining: 1m 5s
60:	learn: 0.1348312	total: 8.83s	remaining: 1m 3s
70:	learn: 0.1315098	total: 10.2s	remaining: 1m 1s
80:	learn: 0.1282880	total: 11.7s	remaining: 1m
90:	learn: 0.1259588	total: 13.1s	remaining: 58.7s
100:	learn: 0.1235569	total: 14.5s	remaining: 57.3s
110:	learn: 0.1212450	total: 15.9s	remaining: 55.8s
120:	learn: 0.1195540	total: 17.3s	remaining: 54.2s
130:	learn: 0.1176325	total: 18.7s	remaining: 52.7s
140:	learn: 0.1163747	total: 20.1s	remaining: 51.3s
150:	learn: 0.1148145	total: 21.5s	remaining: 49.8s
160:	learn: 0.1130164	total: 22.9s	remaining: 48.3s
170:	learn: 0.1120260	total: 24.3s	remaining: 46.8s
180:	learn: 0.1101967	total: 25.8s	remaining: 45.4s
190:	learn: 0.1093257	t

In [23]:
predict = model.predict(tf_idf_test)
f1_score(y_test, predict)

0.7641114982578397

### XGBRegressor

In [24]:
xg_reg = xgb.XGBClassifier(
    objective ='binary:logistic', 
    learning_rate = 0.7,
)
xg_reg.fit(tf_idf, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.7, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [25]:
predict = xg_reg.predict(tf_idf_test)
f1_score(y_test, predict)

0.7564856451054998

### Проверка на адекватность

In [26]:
def coin_model(data):
    # Случайно выбирает класс
    return np.random.randint(2, size=data.shape[0])

def one_model(data):
    # Выбирает всегда один класс
    return np.ones(data.shape[0])

In [27]:
predict_1 = coin_model(X_test)
predict_2 = one_model(X_test)
print('Случайная модель', f1_score(y_test, predict_1))
print('Константная модель', f1_score(y_test, predict_2))

Случайная модель 0.1647095663065602
Константная модель 0.1845331209647601


# 3. Выводы

Лучшая модель - CatBoostRegressor. <br>
Модель даекватная, т.к. случайная и константная модели намного хуже.